Copyright (c) Microsoft Corporation. All rights reserved.  
Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/NotebookVM/how-to-use-azureml/machine-learning-pipelines/intro-to-pipelines/aml-pipelines-with-automated-machine-learning-step.png)

# Azure Machine Learning Pipeline with AutoMLStep (Udacity Course 2)
This notebook demonstrates the use of AutoMLStep in Azure Machine Learning Pipeline.

## Introduction
In this example we showcase how you can use AzureML Dataset to load data for AutoML via AML Pipeline. 

If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, make sure you have executed the [configuration](https://aka.ms/pl-config) before running this notebook.

In this notebook you will learn how to:
1. Create an `Experiment` in an existing `Workspace`.
2. Create or Attach existing AmlCompute to a workspace.
3. Define data loading in a `TabularDataset`.
4. Configure AutoML using `AutoMLConfig`.
5. Use AutoMLStep
6. Train the model using AmlCompute
7. Explore the results.
8. Test the best fitted model.

## Azure Machine Learning and Pipeline SDK-specific imports

In [85]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

StatementMeta(248ac706-8c11-4524-99b9-ff04fe22c28c, 0, 171, Finished, Available)

[2023-12-19T10:44:17.184165] post artifact meta request successfully.
[2023-12-19T10:44:17.327560] get artifact sas uri successfully. origin is ExperimentRun artifact path is logs/azureml/driver/stderr
[2023-12-19T10:44:17.506341] post artifact meta request successfully.
[2023-12-19T10:44:17.644772] get artifact sas uri successfully. origin is ExperimentRun artifact path is logs/azureml/driver/stdout
Successfully add logs to artifact services.
Start to mount artifact container with mount point /AmlJobLogs/dcid.11fd0a83-7b0a-466e-ad05-67217b74fe89
Successfully mount artifact container to path:  /synfs/notebook/AmlJobLogs/dcid.11fd0a83-7b0a-466e-ad05-67217b74fe89
Starting log upload threads
SDK version: 1.54.0


## Initialize Workspace
Initialize a workspace object from persisted configuration. Make sure the config file is present at .\config.json

In [86]:
ws = Workspace.from_config(path='Users/odl_user_245916/config.json')
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

StatementMeta(248ac706-8c11-4524-99b9-ff04fe22c28c, 0, 172, Finished, Available)

quick-starts-ws-245916
aml-quickstarts-245916
westeurope
f9d5a085-54dc-4215-9ba6-dad5d86e60a0


## Create an Azure ML experiment
Let's create an experiment named "automlstep-classification" and a folder to hold the training scripts. The script runs will be recorded under the experiment in Azure.

The best practice is to use separate folders for scripts and its dependent files for each step and specify that folder as the `source_directory` for the step. This helps reduce the size of the snapshot created for the step (only the specific folder is snapshotted). Since changes in any files in the `source_directory` would trigger a re-upload of the snapshot, this helps keep the reuse of the step when there are no changes in the `source_directory` of the step.

*Udacity Note:* There is no need to create an Azure ML experiment, this needs to re-use the experiment that was already created


In [87]:
# Choose a name for the run history container in the workspace.
# NOTE: update these to match your existing experiment name
experiment_name = 'ml-experiment-1'
project_folder = './pipeline-project'

experiment = Experiment(ws, experiment_name)
experiment

StatementMeta(248ac706-8c11-4524-99b9-ff04fe22c28c, 0, 173, Finished, Available)

Name,Workspace,Report Page,Docs Page
ml-experiment-1,quick-starts-ws-245916,Link to Azure Machine Learning studio,Link to Documentation


### Create or Attach an AmlCompute cluster
You will need to create a [compute target](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#compute-target) for your AutoML run. In this tutorial, you get the default `AmlCompute` as your training compute resource.

**Udacity Note** There is no need to create a new compute target, it can re-use the previous cluster

In [88]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "auto-ml"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

StatementMeta(248ac706-8c11-4524-99b9-ff04fe22c28c, 0, 174, Finished, Available)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Data

**Udacity note:** Make sure the `key` is the same name as the dataset that is uploaded, and that the description matches. If it is hard to find or unknown, loop over the `ws.datasets.keys()` and `print()` them.
If it *isn't* found because it was deleted, it can be recreated with the link that has the CSV 

In [89]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "Bank_marketingnew"
description_text = "Bank Marketing DataSet for Udacity Course 2"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

StatementMeta(248ac706-8c11-4524-99b9-ff04fe22c28c, 0, 175, Finished, Available)

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
count,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000,32950.000000
mean,40.040212,257.335205,2.561730,962.174780,0.174780,0.076228,93.574243,-40.518680,3.615654,5166.859608
std,10.432313,257.331700,2.763646,187.646785,0.496503,1.572242,0.578636,4.623004,1.735748,72.208448
min,17.000000,0.000000,1.000000,0.000000,0.000000,-3.400000,92.201000,-50.800000,0.634000,4963.600000
25%,32.000000,102.000000,1.000000,999.000000,0.000000,-1.800000,93.075000,-42.700000,1.344000,5099.100000
50%,38.000000,179.000000,2.000000,999.000000,0.000000,1.100000,93.749000,-41.800000,4.857000,5191.000000
75%,47.000000,318.000000,3.000000,999.000000,0.000000,1.400000,93.994000,-36.400000,4.961000,5228.100000
max,98.000000,4918.000000,56.000000,999.000000,7.000000,1.400000,94.767000,-26.900000,5.045000,5228.100000


### Review the Dataset Result

You can peek the result of a TabularDataset at any range using `skip(i)` and `take(j).to_pandas_dataframe()`. Doing so evaluates only `j` records for all the steps in the TabularDataset, which makes it fast even against large datasets.

`TabularDataset` objects are composed of a list of transformation steps (optional).

In [90]:
dataset.take(5).to_pandas_dataframe()

StatementMeta(248ac706-8c11-4524-99b9-ff04fe22c28c, 0, 176, Finished, Available)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,57,technician,married,high.school,no,no,yes,cellular,may,mon,...,1,999,1,failure,-1.8,92.893,-46.2,1.299,5099.1,no
1,55,unknown,married,unknown,unknown,yes,no,telephone,may,thu,...,2,999,0,nonexistent,1.1,93.994,-36.4,4.860,5191.0,no
2,33,blue-collar,married,basic.9y,no,no,no,cellular,may,fri,...,1,999,1,failure,-1.8,92.893,-46.2,1.313,5099.1,no
3,36,admin.,married,high.school,no,no,no,telephone,jun,fri,...,4,999,0,nonexistent,1.4,94.465,-41.8,4.967,5228.1,no
4,27,housemaid,married,high.school,no,yes,no,cellular,jul,fri,...,2,999,0,nonexistent,1.4,93.918,-42.7,4.963,5228.1,no


## Train
This creates a general AutoML settings object.
**Udacity notes:** These inputs must match what was used when training in the portal. `label_column_name` has to be `y` for example.

In [91]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'
}
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="y",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

StatementMeta(248ac706-8c11-4524-99b9-ff04fe22c28c, 0, 177, Finished, Available)

#### Create Pipeline and AutoMLStep

You can define outputs for the AutoMLStep using TrainingOutput.

In [92]:
from azureml.pipeline.core import PipelineData, TrainingOutput

ds = ws.get_default_datastore()
metrics_output_name = 'metrics_output'
best_model_output_name = 'best_model_output'

metrics_data = PipelineData(name='metrics_data',
                           datastore=ds,
                           pipeline_output_name=metrics_output_name,
                           training_output=TrainingOutput(type='Metrics'))
model_data = PipelineData(name='model_data',
                           datastore=ds,
                           pipeline_output_name=best_model_output_name,
                           training_output=TrainingOutput(type='Model'))

StatementMeta(248ac706-8c11-4524-99b9-ff04fe22c28c, 0, 178, Finished, Available)

Create an AutoMLStep.

In [93]:
automl_step = AutoMLStep(
    name='automl_module',
    automl_config=automl_config,
    outputs=[metrics_data, model_data],
    allow_reuse=True)

StatementMeta(248ac706-8c11-4524-99b9-ff04fe22c28c, 0, 179, Finished, Available)

In [94]:
from azureml.pipeline.core import Pipeline
pipeline = Pipeline(
    description="pipeline_with_automlstep",
    workspace=ws,    
    steps=[automl_step])

StatementMeta(248ac706-8c11-4524-99b9-ff04fe22c28c, 0, 180, Finished, Available)

In [95]:
pipeline_run = experiment.submit(pipeline)

StatementMeta(248ac706-8c11-4524-99b9-ff04fe22c28c, 0, 181, Finished, Available)

Created step automl_module [6eb57c8b][644e007d-f9dc-48f2-a218-529717fc2f87], (This step is eligible to reuse a previous run's output)
Submitted PipelineRun be1556db-e803-491e-b817-cdb17b01e2e6
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/be1556db-e803-491e-b817-cdb17b01e2e6?wsid=/subscriptions/f9d5a085-54dc-4215-9ba6-dad5d86e60a0/resourcegroups/aml-quickstarts-245916/workspaces/quick-starts-ws-245916&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254


In [96]:
from azureml.widgets import RunDetails
RunDetails(pipeline_run).show()

StatementMeta(248ac706-8c11-4524-99b9-ff04fe22c28c, 0, 182, Finished, Available)

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

In [97]:
pipeline_run.wait_for_completion()

StatementMeta(248ac706-8c11-4524-99b9-ff04fe22c28c, 0, 183, Finished, Available)

PipelineRunId: be1556db-e803-491e-b817-cdb17b01e2e6
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/be1556db-e803-491e-b817-cdb17b01e2e6?wsid=/subscriptions/f9d5a085-54dc-4215-9ba6-dad5d86e60a0/resourcegroups/aml-quickstarts-245916/workspaces/quick-starts-ws-245916&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254
PipelineRun Status: Running


StepRunId: 6ec12b75-580b-40e6-b2aa-cfa59828459e
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/6ec12b75-580b-40e6-b2aa-cfa59828459e?wsid=/subscriptions/f9d5a085-54dc-4215-9ba6-dad5d86e60a0/resourcegroups/aml-quickstarts-245916/workspaces/quick-starts-ws-245916&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

StepRun(automl_module) Execution Summary
StepRun( automl_module ) Status: Finished
{'runId': '6ec12b75-580b-40e6-b2aa-cfa59828459e', 'target': 'auto-ml', 'status': 'Completed', 'startTimeUtc': '2023-12-19T10:45:00.770224Z', 'endTimeUtc': '2023-12-19T10:45:00.770224Z', 'services': {}, 'properties': {'azureml.isreused': 'true

'Finished'

## Examine Results

### Retrieve the metrics of all child runs
Outputs of above run can be used as inputs of other steps in pipeline. In this tutorial, we will examine the outputs by retrieve output data and running some tests.

In [98]:
metrics_output = pipeline_run.get_pipeline_output(metrics_output_name)
num_file_downloaded = metrics_output.download('.', show_progress=True)

StatementMeta(248ac706-8c11-4524-99b9-ff04fe22c28c, 0, 184, Finished, Available)

Path already exists. Skipping download for ./azureml/ab140334-971d-4bff-a6bf-37f735d53197/metrics_data


In [99]:
import json
with open(metrics_output._path_on_datastore) as f:
    metrics_output_result = f.read()
    
deserialized_metrics_output = json.loads(metrics_output_result)
df = pd.DataFrame(deserialized_metrics_output)
df

StatementMeta(248ac706-8c11-4524-99b9-ff04fe22c28c, 0, 185, Finished, Available)

,ab140334-971d-4bff-a6bf-37f735d53197_0,ab140334-971d-4bff-a6bf-37f735d53197_23,ab140334-971d-4bff-a6bf-37f735d53197_16,ab140334-971d-4bff-a6bf-37f735d53197_31,ab140334-971d-4bff-a6bf-37f735d53197_25,ab140334-971d-4bff-a6bf-37f735d53197_29,ab140334-971d-4bff-a6bf-37f735d53197_27,ab140334-971d-4bff-a6bf-37f735d53197_22,ab140334-971d-4bff-a6bf-37f735d53197_26,ab140334-971d-4bff-a6bf-37f735d53197_24,...,ab140334-971d-4bff-a6bf-37f735d53197_14,ab140334-971d-4bff-a6bf-37f735d53197_21,ab140334-971d-4bff-a6bf-37f735d53197_18,ab140334-971d-4bff-a6bf-37f735d53197_42,ab140334-971d-4bff-a6bf-37f735d53197_44,ab140334-971d-4bff-a6bf-37f735d53197_28,ab140334-971d-4bff-a6bf-37f735d53197_30,ab140334-971d-4bff-a6bf-37f735d53197_37,ab140334-971d-4bff-a6bf-37f735d53197_7,ab140334-971d-4bff-a6bf-37f735d53197_35
average_precision_score_weighted,[0.9535822261506017],[0.9352160969860712],[0.9364341205088045],[0.9374831766451989],[0.9473144877813431],[0.9514215544812085],[0.9483550119623401],[0.9517474249253096],[0.9511014381205041],[0.9532327144121392],...,[0.9517044339373133],[0.9512078722916548],[0.9519080952569361],[0.9541358029182306],[0.9521945653456325],[0.9499340331794855],[0.9526950938003461],[0.9491757264778352],[0.9480633299225389],[0.9513391767199155]
recall_score_macro,[0.7445642005975768],[0.8389043562342664],[0.8539734406229913],[0.6658386542853809],[0.6768857657817863],[0.6929870870820807],[0.6965154015860049],[0.7474451094476768],[0.7339070143948192],[0.7207468041871123],...,[0.7056707733857926],[0.7281572371431164],[0.7462730180958679],[0.7467615824483604],[0.733394369145332],[0.7308672642433874],[0.6379682576730074],[0.750802542201772],[0.6863829010812322],[0.7305255007437292]
recall_score_micro,[0.9101669195751139],[0.7517450682852808],[0.8437025796661608],[0.8986342943854325],[0.9077389984825494],[0.9132018209408195],[0.9089529590288316],[0.9110773899848255],[0.9101669195751139],[0.9162367223065251],...,[0.9125948406676783],[0.9104704097116844],[0.9132018209408195],[0.9098634294385433],[0.9092564491654022],[0.9110773899848255],[0.9059180576631259],[0.906525037936267],[0.9119878603945372],[0.9104704097116844]
balanced_accuracy,[0.7445642005975768],[0.8389043562342664],[0.8539734406229913],[0.6658386542853809],[0.6768857657817863],[0.6929870870820807],[0.6965154015860049],[0.7474451094476768],[0.7339070143948192],[0.7207468041871123],...,[0.7056707733857926],[0.7281572371431164],[0.7462730180958679],[0.7467615824483604],[0.733394369145332],[0.7308672642433874],[0.6379682576730074],[0.750802542201772],[0.6863829010812322],[0.7305255007437292]
norm_macro_recall,[0.48912840119515355],[0.6778087124685328],[0.7079468812459826],[0.33167730857076183],[0.35377153156357255],[0.38597417416416135],[0.3930308031720098],[0.4948902188953537],[0.4678140287896384],[0.4414936083742247],...,[0.4113415467715853],[0.45631447428623284],[0.49254603619173576],[0.4935231648967209],[0.4667887382906639],[0.4617345284867749],[0.2759365153460147],[0.501605084403544],[0.37276580216246447],[0.4610510014874585]
matthews_correlation,[0.5216286298277554],[0.44833994948953654],[0.5238288494614208],[0.4064166205228067],[0.4517296920380605],[0.48864489691810753],[0.47422181829067633],[0.5270365568045118],[0.5120524394053735],[0.5229195744551492],...,[0.4963151352876262],[0.5079177049128708],[0.5328108142609521],[0.5226778010101003],[0.5086578080065534],[0.5123580976029641],[0.408491687757675],[0.5166443905575122],[0.47805766735773875],[0.5100263931184434]
weighted_accuracy,[0.9512815952194833],[0.7301057747063945],[0.8411526027126678],[0.9564311548067598],[0.9650536116878642],[0.9678751826501844],[0.9616954582031879],[0.9517028590639043],[0.9539274862816189],[0.9647715810627646],...,[0.9639684893724584],[0.9557338389725855],[0.9546457273395061],[0.9503572062158955],[0.9529182466456118],[0.9558186896050043],[0.9724427450812216],[0.9451867214049058],[0.9679994692811393],[0.9551458631809997]
recall_score_weighted,[0.9101669195751139],[0.7517450682852808],[0.843702579666

### Retrieve the Best Model

In [100]:
# Retrieve best model from Pipeline Run
best_model_output = pipeline_run.get_pipeline_output(best_model_output_name)
num_file_downloaded = best_model_output.download('.', show_progress=True)

StatementMeta(248ac706-8c11-4524-99b9-ff04fe22c28c, 0, 186, Finished, Available)

Path already exists. Skipping download for ./azureml/ab140334-971d-4bff-a6bf-37f735d53197/model_data


In [108]:
import os
curr_dir = os.listdir('.')
print(curr_dir)
#print(num_file_downloaded)

StatementMeta(248ac706-8c11-4524-99b9-ff04fe22c28c, 0, 194, Finished, Available)

['Users', 'azureml', 'azureml-logs', 'automl_errors.log', 'Logs', 'pipeline-project']


In [104]:
'''import pickle
#import azureml.training

#with open(best_model_output./azureml/ab140334-971d-4bff-a6bf-37f735d53197/model_data, "rb" ) as f:
with open(best_model_output,"rb") as f:    
    best_model = pickle.load(f)
best_model'''

StatementMeta(248ac706-8c11-4524-99b9-ff04fe22c28c, 0, 190, Finished, Available)

TypeError: expected str, bytes or os.PathLike object, not PortDataReference

In [102]:
#best_model.steps

StatementMeta(248ac706-8c11-4524-99b9-ff04fe22c28c, 0, 188, Finished, Available)

NameError: name 'best_model' is not defined

### Test the Model
#### Load Test Data
For the test data, it should have the same preparation step as the train data. Otherwise it might get failed at the preprocessing step.

In [105]:
dataset_test = Dataset.Tabular.from_delimited_files(path='https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv')
df_test = dataset_test.to_pandas_dataframe()
df_test = df_test[pd.notnull(df_test['y'])]

y_test = df_test['y']
X_test = df_test.drop(['y'], axis=1)

StatementMeta(248ac706-8c11-4524-99b9-ff04fe22c28c, 0, 191, Finished, Available)

#### Testing Our Best Fitted Model

We will use confusion matrix to see how our model works.

In [ ]:
from sklearn.metrics import confusion_matrix
ypred = best_model.predict(X_test)
cm = confusion_matrix(y_test, ypred)

StatementMeta(, , , Cancelled, )

In [ ]:
# Visualize the confusion matrix
pd.DataFrame(cm).style.background_gradient(cmap='Blues', low=0, high=0.9)

StatementMeta(, , , Cancelled, )

## Publish and run from REST endpoint

Run the following code to publish the pipeline to your workspace. In your workspace in the portal, you can see metadata for the pipeline including run history and durations. You can also run the pipeline manually from the portal.

Additionally, publishing the pipeline enables a REST endpoint to rerun the pipeline from any HTTP library on any platform.


In [ ]:
published_pipeline = pipeline_run.publish_pipeline(
    name="Bankmarketing Train", description="Training bankmarketing pipeline", version="1.0")

published_pipeline


StatementMeta(, , , Cancelled, )

Authenticate once again, to retrieve the `auth_header` so that the endpoint can be used

In [ ]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()



StatementMeta(, , , Cancelled, )

Get the REST url from the endpoint property of the published pipeline object. You can also find the REST url in your workspace in the portal. Build an HTTP POST request to the endpoint, specifying your authentication header. Additionally, add a JSON payload object with the experiment name and the batch size parameter. As a reminder, the process_count_per_node is passed through to ParallelRunStep because you defined it is defined as a PipelineParameter object in the step configuration.

Make the request to trigger the run. Access the Id key from the response dict to get the value of the run id.


In [ ]:
import requests

rest_endpoint = published_pipeline.endpoint
response = requests.post(rest_endpoint, 
                         headers=auth_header, 
                         json={"ExperimentName": "pipeline-rest-endpoint"}
                        )

StatementMeta(, , , Cancelled, )

In [ ]:
try:
    response.raise_for_status()
except Exception:    
    raise Exception("Received bad response from the endpoint: {}\n"
                    "Response Code: {}\n"
                    "Headers: {}\n"
                    "Content: {}".format(rest_endpoint, response.status_code, response.headers, response.content))

run_id = response.json().get('Id')
print('Submitted pipeline run: ', run_id)

StatementMeta(, , , Cancelled, )

Use the run id to monitor the status of the new run. This will take another 10-15 min to run and will look similar to the previous pipeline run, so if you don't need to see another pipeline run, you can skip watching the full output.

In [ ]:
from azureml.pipeline.core.run import PipelineRun
from azureml.widgets import RunDetails

published_pipeline_run = PipelineRun(ws.experiments["pipeline-rest-endpoint"], run_id)
RunDetails(published_pipeline_run).show()

StatementMeta(, , , Cancelled, )